**TASK 3: Statement-based query**

In [1]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2024-2025/master/Assignment4/course_materials"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00


Importamos example3.rdf en nuestro grafo

In [2]:
from rdflib import Graph, Namespace, Literal
g = Graph()
g.parse(github_storage+"/rdf/example3.rdf", format="xml")

<Graph identifier=N7cbd19d2fd924debb3a0f35a22ff5a7b (<class 'rdflib.graph.Graph'>)>

Listamos todos los recursos con la propiedad VCARD:FN

In [3]:
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

for s,p,o in g.triples((None, VCARD.FN, None)):
  print(s)

http://somewhere#JaneSmith
http://somewhere#JohnSmith
http://somewhere#SarahJones
http://somewhere#MattJones


Listamos ahora el nombre completo (VCARD:FN) de estos recursos

In [4]:
for s,p,o in g.triples((None, VCARD.FN, None)):
  print(o)

Jane Smith
John Smith
Sarah Jones
Matt Jones


Listamos ahora todos los recursos que tienen el nombre de familia "Smith", debemos tener en cuenta el tipo de dato de nuestro literales a la hora de realizar búsquedas. En nuestro ejemplo los literales están tipados como XSD String (http://www.w3.org/2001/XMLSchema#string) y al buscar debemos indicarlo, de lo contrario la comparación fallará aunque las cadenas de texto sean iguales.

In [9]:
from rdflib import XSD
for t in g.triples((None, VCARD.Family, Literal('Smith', datatype=XSD.string))):
  print(t)

http://somewhere#JaneSmith
http://somewhere#JohnSmith


In [11]:
print(Literal(1, datatype=XSD.number).eq(Literal("1", datatype=XSD.string)))
print(Literal(1, datatype=XSD.integer).eq(Literal(1, datatype=XSD.string)))
print(Literal(1).eq(Literal(1, datatype=XSD.number)))
print(Literal(1, datatype=XSD.number).eq(Literal(1, datatype=XSD.number)))
print(Literal(1).datatype)
print(Literal("1").datatype)

False
False
True
True
http://www.w3.org/2001/XMLSchema#integer
None


In [14]:
print(Literal(-1.1).datatype)

http://www.w3.org/2001/XMLSchema#double


Listamos todos los recursos que contienen un email

In [15]:
#from rdflib import FOAF
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
for s,p,o in g.triples((None, FOAF.email, None)):
  print(s,p,o)

http://somewhere#JaneSmith http://xmlns.com/foaf/0.1/email jSmith@somewhere.com
http://somewhere#SarahJones http://xmlns.com/foaf/0.1/email sJones@somewhere.com


Listamos ahora todos los sujetos que conocen a "Jane Smith" (VCARD:FN) por su nombre de pila (VCARD:Given). Cuando sepamos que vamos a obtener un solo resultado podemos emplear la función value indicando cada parámetro conocido (solo uno debe ser None).

In [16]:
janeSmithURI = g.value(subject=None, predicate=VCARD.FN, object=Literal("Jane Smith"))

knows = g.triples((None, FOAF.knows, janeSmithURI))

for s,p,o in knows:

  givenName = g.value(subject=s, predicate=VCARD.Given, object=None)

  print(givenName)

John
Matt
